# NET-seq TASEP Speed Benchmark

This notebook compares baseline Python TASEP code and optimized TASEP code using Numba JIT using the exact baseline default simulation parameters (geneLength=3075, uniform dwell profile).

Runs are executed in a machine with an Intel(R) Core(TM) i7-10750H CPU. The MATLAB code (sjkimlab_NETSEQ_TASEP.m) takes approximately ~4s per run for this parameter set.

In [ ]:
import os
import shutil
import subprocess
import sys
import time
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

cwd = Path.cwd().resolve()
if (cwd / 'sjkimlab_NETSEQ_TASEP.py').exists():
    module_dir = cwd
elif (cwd / 'sjkimlab_NETSEQ_TASEP' / 'sjkimlab_NETSEQ_TASEP.py').exists():
    module_dir = cwd / 'sjkimlab_NETSEQ_TASEP'
else:
    raise FileNotFoundError('Could not locate sjkimlab_NETSEQ_TASEP.py from current working directory')

if str(module_dir) not in sys.path:
    sys.path.insert(0, str(module_dir))

from sjkimlab_NETSEQ_TASEP import netseq_tasep_function
from netseq_tasep_fast import (
    SNAPSHOTS,
    _worker,
    netseq_tasep_fast,
)

SEED = 42
N_RUNS_BASELINE = 12
N_RUNS_SWEEP = 12

# Match the default parameter set used inside netseq_tasep_function exactly.
PARAMS = {
    'RNAPSpeed': 19,
    'ribospeed': 19,
    'kLoading': 1 / 20,
    'kRiboLoading': 0,
    'KRutLoading': 0.13,
    'simtime': 2000,
    'glutime': 1600,
    'geneLength': 3075,
    'RNAP_dwellTimeProfile': np.ones(3075, dtype=float),
}


def run_fast_with_params(n_runs: int, seed: int | None = None, n_workers: int | None = None) -> np.ndarray:
    if n_runs <= 0:
        raise ValueError('n_runs must be positive')

    base_seed = int(seed) if seed is not None else int(np.random.SeedSequence().generate_state(1)[0])
    seeds = [base_seed + i for i in range(n_runs)]

    if n_workers is None:
        n_workers = os.cpu_count() or 1
    n_workers = max(1, int(n_workers))

    if n_workers == 1:
        outputs = [_worker((PARAMS, s)) for s in seeds]
        total = np.zeros_like(outputs[0], dtype=float)
        for out in outputs:
            total += out
        return total / float(n_runs)

    worker_args = [(PARAMS, s) for s in seeds]
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        outputs = list(executor.map(_worker, worker_args))

    total = np.zeros_like(outputs[0], dtype=float)
    for out in outputs:
        total += out
    return total / float(n_runs)


print(
    f'Configured baseline-default parameters: '
    f"geneLength={PARAMS['geneLength']}, "
    f"uniform_dwell={np.allclose(PARAMS['RNAP_dwellTimeProfile'], 1.0)}, "
    f"seed={SEED}"
)

Configured baseline-default parameters: geneLength=3075, uniform_dwell=True, seed=42


In [2]:
get_ipython().run_line_magic("matplotlib", "inline")
plt.switch_backend("module://matplotlib_inline.backend_inline")

In [3]:
# Dedicated JIT warm-up (single-process compile cost)
t0 = time.perf_counter()
_ = netseq_tasep_fast(PARAMS, seed=SEED)
jit_compile_time = time.perf_counter() - t0
print(f"JIT compile time: {jit_compile_time:.1f}s")
print("Note: with cache=True, subsequent sessions will skip JIT compilation.")

JIT compile time: 2.0s
Note: with cache=True, subsequent sessions will skip JIT compilation.


In [4]:
# Baseline single-run timing
rng = np.random.default_rng(SEED)
t0 = time.perf_counter()
baseline_single = netseq_tasep_function(PARAMS, rng)
t_baseline_single = time.perf_counter() - t0

net_single = np.asarray(baseline_single["NETseq"], dtype=float)
cols_single = [t - 1 for t in SNAPSHOTS if 1 <= t <= net_single.shape[1]]
baseline_single_sum = net_single[:, cols_single].sum(axis=1)

print(f"Baseline single run: {t_baseline_single:.3f}s")

Baseline single run: 7.534s


In [5]:
# Baseline n_runs=10 sequential timing
baseline_runs = []
t0 = time.perf_counter()
for i in range(N_RUNS_BASELINE):
    rng = np.random.default_rng(SEED + i)
    out = netseq_tasep_function(PARAMS, rng)
    net = np.asarray(out["NETseq"], dtype=float)
    cols = [t - 1 for t in SNAPSHOTS if 1 <= t <= net.shape[1]]
    baseline_runs.append(net[:, cols].sum(axis=1))
t_baseline_nruns = time.perf_counter() - t0
baseline_avg_nruns = np.mean(np.vstack(baseline_runs), axis=0)

print(f"Baseline n_runs={N_RUNS_BASELINE}: {t_baseline_nruns:.3f}s")

Baseline n_runs=8: 48.560s


In [6]:
# Fast single-run timing
t0 = time.perf_counter()
fast_single = netseq_tasep_fast(PARAMS, seed=SEED)
t_fast_single = time.perf_counter() - t0
fast_single_sum = np.asarray(fast_single["NETseq_sum"], dtype=float)

print(f"Fast single run: {t_fast_single:.3f}s")

Fast single run: 0.062s


In [7]:
# Worker-count sweep (ThreadPoolExecutor with nogil=True Numba kernels)
worker_candidates = [1, 2, 4, os.cpu_count() or 1]
worker_values = []
for w in worker_candidates:
    if w not in worker_values:
        worker_values.append(w)

worker_times = {}
for w in worker_values:
    _ = run_fast_with_params(n_runs=1, seed=SEED, n_workers=w)  # untimed pre-warm
    t0 = time.perf_counter()
    _ = run_fast_with_params(n_runs=N_RUNS_SWEEP, seed=SEED, n_workers=w)
    worker_times[w] = time.perf_counter() - t0

print('Worker sweep timings (ThreadPoolExecutor, nogil=True):')
for w in worker_values:
    print(f'  n_workers={w}: {worker_times[w]:.3f}s')

Worker sweep timings (ThreadPoolExecutor, nogil=True):
  n_workers=1: 0.519s
  n_workers=2: 0.176s
  n_workers=4: 0.126s
  n_workers=12: 0.127s
